In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models

2025-05-25 14:18:29.859042: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748182709.884449    2140 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748182709.892320    2140 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-25 14:18:29.921320: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from tensorflow.keras.layers import Dropout, BatchNormalization

def build_cnn_lstm_model():
    inputs = tf.keras.Input(shape=(24, 1))  # Shape: (timesteps, features)

    # CNN part
    x = layers.Conv1D(32, kernel_size=3, padding="same", activation="relu")(inputs)
    x = layers.MaxPooling1D(pool_size=2)(x)

    # LSTM part
    x = layers.LSTM(64, return_sequences=True, activation="relu")(x)
    x = Dropout(0.2)(x)
    x = BatchNormalization()(x)

    x = layers.LSTM(128, return_sequences=False, activation="relu")(x)  # Final LSTM layer
    x = Dropout(0.2)(x)
    x = BatchNormalization()(x)

    # Dense layers
    x = layers.Dense(256, activation="relu")(x)
    x = Dropout(0.2)(x)
    x = BatchNormalization()(x)

    x = layers.Dense(512, activation="relu")(x)
    outputs = layers.Dense(1)(x)  # Regression output (e.g., BPM)

    model = tf.keras.Model(inputs, outputs)
    return model


In [3]:
callback=tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=0,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=False,
    start_from_epoch=0
)

In [4]:
# Check TensorFlow version and GPU availability
print("TensorFlow version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))

TensorFlow version: 2.18.0
GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [5]:
model=build_cnn_lstm_model()
# model1=build_cnn_lstm_model()

I0000 00:00:1748182714.122778    2140 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2248 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5


In [6]:
import numpy as np
import pandas as pd
import os

In [7]:
# Load the datasets
X1 = np.load("input_media.npy")   # Shape: (samples1, 24, 1)
Y1 = np.load("out_media.npy")     # Shape: (samples1, 24)
X2 = np.load("input.npy") # Shape: (samples2, 24, 1)
Y2 = np.load("out.npy")   # Shape: (samples2, 24)

# Concatenate along the first axis (number of samples)
X_combined = np.concatenate((X1, X2), axis=0)
Y_combined = np.concatenate((Y1, Y2), axis=0)

In [9]:
X_combined

array([[[129.51707451],
        [129.75986121],
        [129.75986121],
        ...,
        [130.60593834],
        [130.00177458],
        [130.01250866]],

       [[129.98844356],
        [129.97351108],
        [130.45717293],
        ...,
        [128.9816    ],
        [128.2174466 ],
        [127.99740305]],

       [[127.97649758],
        [128.05176593],
        [128.16763331],
        ...,
        [128.49357484],
        [127.99792244],
        [128.56791369]],

       ...,

       [[102.29422963],
        [101.78716716],
        [101.78716716],
        ...,
        [100.57082101],
        [100.57082101],
        [ 99.99666251]],

       [[ 99.99666251],
        [ 99.24707409],
        [ 99.24707409],
        ...,
        [ 97.53497031],
        [ 98.18329085],
        [ 98.18329085]],

       [[ 98.18329085],
        [ 97.99323596],
        [ 97.99323596],
        ...,
        [ 99.94760233],
        [ 99.84978095],
        [ 99.84978095]]])

In [10]:
Y_combined

array([[ 98],
       [ 98],
       [ 98],
       ...,
       [100],
       [ 99],
       [ 99]])

In [12]:
model.compile(optimizer="adam",metrics=["mae"],loss="mae")
# model1.compile(optimizer="adam",metrics=["mae"],loss="mae")

In [13]:
model.fit(X_combined,Y_combined,epochs=200,callbacks=callback,validation_split=0.3)
# model1.fit(X,Y,epochs=200,callbacks=callback,validation_split=0.2)

Epoch 1/200


I0000 00:00:1748182761.310039    2210 service.cc:148] XLA service 0x7fae500cfe40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1748182761.310318    2210 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
2025-05-25 14:19:21.477329: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1748182762.308726    2210 cuda_dnn.cc:529] Loaded cuDNN version 90300


 5/95 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - loss: 95.4816 - mae: 95.4816

I0000 00:00:1748182771.061888    2210 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


95/95 ━━━━━━━━━━━━━━━━━━━━ 24s 102ms/step - loss: 80.9505 - mae: 80.9505 - val_loss: 147.6571 - val_mae: 147.6571
Epoch 2/200
95/95 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 17.4005 - mae: 17.4005 - val_loss: 16.5829 - val_mae: 16.5829
Epoch 3/200
95/95 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 15.6528 - mae: 15.6528 - val_loss: 14.1943 - val_mae: 14.1943
Epoch 4/200
95/95 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 15.9648 - mae: 15.9648 - val_loss: 13.9922 - val_mae: 13.9922
Epoch 5/200
95/95 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 16.0691 - mae: 16.0691 - val_loss: 15.7440 - val_mae: 15.7440


In [ ]:
X.shape

In [ ]:
Y

In [ ]:
# model.save("bpm_model_media.keras")